In [6]:
import os
import sys
from logzero import logger
from matplotlib import pyplot as plt
from itertools import cycle
from scipy import stats
import random

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from enum import IntEnum

from PIL import Image 
Image.MAX_IMAGE_PIXELS = 1171568999
                                               

In [7]:
records = pd.read_csv('/lustre/nvme/kiviaho/tfr_KF_1_2_3_4_in_5.tsv', sep = '\t')
files = list(dict.fromkeys(records['TFRecords'])) # This preserves the order, unlike set !!
    
random.seed(42)
random.shuffle(files)

In [8]:
sample_info = pd.read_csv('/lustre/scratch/kiviaho/myoma/myoma-new/sample_information_KF_1_2_3_4_in_5.tsv',sep='\t')



In [9]:
for sample in files[8151:8152]:
    print("TFRecord: "+sample)
    subset = sample_info[sample_info['TFRecords'] == sample]
    size = len(subset['Tile'])
    i=0
    for path in subset['Tile']:
        try:
            img = plt.imread(path)
        except:
            print(path + ' is problematic')
        if i % 800 == 0:
            print('Tile '+ str(i) + '/'+ str(size))
        i+=1


TFRecord: /lustre/nvme/kiviaho/tfrecords/KF_1_in_5_1834.tfr
Tile 0/1600
Tile 800/1600


In [126]:
# DOWNLOAD THE SAMPLE INFORMATION MATRICES
results_file = 'binary_label_KF-1-4_2_epochs_100_percent_21461887_at_2022-05-22_01:07:03_598px.h5_new_dataset_binary_prediction_results.csv'
results = pd.read_csv('/lustre/scratch/kiviaho/myoma/myoma-new/false-IHC-stain/'+results_file)

save_path = '/lustre/scratch/kiviaho/myoma/myoma-new/tiling-dir/side-by-side-prediction-plots-alt/'

# Convert labels to 0-1 binary (wt/mut)
results.replace([2,3],1,inplace=True) 

results = results[:len(results)-1]
results = results.rename(columns={'0': 0, '1': 1})

# Tile coordinates are in original image scale and the jpg is down sampled in order to fit into memory.
# Tile coordinates have to be divided by this integer
downsample_lvl = 16
downsampled_tile_size = np.ceil(598/downsample_lvl)



In [127]:
new_samples = np.unique(results['Sample'])[np.where(['m' in s for s in np.unique(results['Sample'])])]

for sample_name in new_samples:
#    sample_name = 'My6146m1'
    img_name = sample_name+'.jpg'
    img = plt.imread('/lustre/scratch/kiviaho/myoma/myoma-new/tiling-dir/Slides-20220326-myoma-JPG-16-downsample/'+img_name)
    img_height,img_width = img.shape[0],img.shape[1]

    df_subset = results[results['Sample']== sample_name]
    df_subset = df_subset.reset_index(drop = True)

    # Splitting rules depend on sample naming schemes
    # Old samples are of form My_6xxx_Tx
    # New samples are of form My6xxxmx

    # Getting the coordinates
    tiles = [s.split('/')[len(s.split('/'))-1].strip('.jpg') for s in df_subset['Path']]
    if 'm' in img_name:
        tile_coords = np.array([s.split('_')[2:4] for s in tiles]).astype(np.int32)
    elif 'T' in img_name:
        tile_coords = np.array([s.split('_')[4:6] for s in tiles]).astype(np.int32)

    tile_coords = np.divide(tile_coords,downsample_lvl).astype(np.int32)
    df_subset['x_coord'] = tile_coords[:,0]
    df_subset['y_coord'] = tile_coords[:,1]

    # Create image mask

    mask = np.ones(img.shape,dtype=np.float64)
    for idx in range(len(df_subset)):
        x_coord = df_subset['x_coord'][idx]
        y_coord = df_subset['y_coord'][idx]
        conf = df_subset[1][idx]

        tile = mask[y_coord:int(y_coord+tile_size),x_coord:int(x_coord+tile_size)]
        new_tile = np.minimum(tile,np.multiply(np.ones(tile.shape),(1,1-conf,1)))
        mask[int(y_coord-tile_size):y_coord,x_coord:int(x_coord+tile_size)] = new_tile



    # Creating the plot
    fig, (ax1, ax2) = plt.subplots(1,2,figsize=(16,16),dpi=600)
    ax1.imshow(img,aspect='auto')
    ax1.set_xlim([0, img_width])
    ax1.set_ylim([img_height,0])

    ax2.imshow(mask,aspect = 'auto')
    ax2.set_xlim([0, img_width])
    ax2.set_ylim([img_height,0])

    plt.savefig(save_path+sample_name+'_side_by_side_prediction.png')
    plt.clf()


/home/ak431480/.conda/envs/myoma-new-env/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


<Figure size 9600x9600 with 0 Axes>

<Figure size 9600x9600 with 0 Axes>

<Figure size 9600x9600 with 0 Axes>

<Figure size 9600x9600 with 0 Axes>

...

<Figure size 9600x9600 with 0 Axes>

<Figure size 9600x9600 with 0 Axes>

<Figure size 9600x9600 with 0 Axes>

<Figure size 9600x9600 with 0 Axes>

<Figure size 9600x9600 with 0 Axes>

In [128]:
def mask_image(mask, image, mask_color=(0, 0, 255,)):
    '''
    mask target region with green color
    '''
    masked = image.copy()
    masked[mask != 0] = np.multiply(mask_color,mask)
    return masked


In [129]:
# # ALTERNATIVE PLOTTING WITH PYPLOT
# # point size 1.6 is approximately 50% overlap

# ax2.scatter(df_subset['x_coord'],df_subset['y_coord'],marker='s',alpha=df_subset[1],s=0.4)
# ax2.set_xlim([0, img_width])
# ax2.set_ylim([img_height,0])